In [1]:
# This script shows you how to make a submission using a few
# useful Python libraries.
# It gets a public leaderboard score of 0.76077.
# Maybe you can tweak it and do better...?

import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

In [2]:
# Load the data
train_df = pd.read_csv('data/train.csv', header=0)
train_df = train_df.drop('id', axis=1)
# train_df = train_df.drop('fac_7', axis=1)
test_df = pd.read_csv('data/test_data.csv', header=0)
train_df

,fac_1,fac_2,fac_3,fac_4,fac_5,fac_6,fac_7,fac_8,poi_1,poi_2,poi_3,size,price_monthly,room_count,total_call,gender
0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1778.0,10038.0,4106.0,9.0000,1500000.0,6.0,72,campur
1,1.0,1.0,0.0,1.0,1.0,1.0,0.0,NaN,4548.0,9332.0,6867.0,12.0000,1500000.0,30.0,56,campur
2,1.0,NaN,1.0,1.0,1.0,1.0,0.0,1.0,5174.0,9021.0,3693.0,12.0000,1600000.0,20.0,109,campur
3,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1490.0,8954.0,2139.0,8.2500,1500000.0,15.0,54,campur
4,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1688.0,8851.0,2145.0,14.8500,2100000.0,10.0,19,campur
5,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1533.0,8715.0,3533.0,9.0000,1350000.0,7.0,5,putri
6,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,969.0,9079.0,2727.0,9.6250,1300000.0,20.0,20,putri
7,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,5200.0,8910.0,3762.0,10.5000,1000000.0,11.0,25,putri
8,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,4835.0,8490.0,3618.0,20.0000,4000000.0,8.0,4,putri
9,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,2880.0,8698.0,2210.0,9.0000,1950000.0,25.0,24,campur


In [3]:
test_df

,id,fac_1,fac_2,fac_3,fac_4,fac_5,fac_6,fac_7,fac_8,poi_1,poi_2,poi_3,size,price_monthly,room_count,total_call
0,3294,0,1,0,0,0,0,0,1,2634,8854,2007,21.00,700000,1,2
1,3295,1,1,1,1,1,1,0,1,6569,3512,7341,12.00,1200000,16,30
2,3296,0,1,1,1,1,0,0,1,10623,883,11250,12.00,700000,8,5
3,3297,0,1,1,1,1,0,0,1,10592,876,11216,12.00,700000,16,6
4,3298,0,1,0,0,1,0,0,1,1623,10204,3931,13.60,700000,6,39
5,3299,1,1,1,1,1,1,0,1,7455,17049,8306,12.00,1600000,8,27
6,3300,1,1,1,1,1,1,0,1,2318,7762,3852,9.00,1300000,11,208
7,3301,0,1,0,1,1,0,0,0,3555,8229,5740,9.00,600000,10,50
8,3302,1,1,1,1,1,0,0,1,3248,8243,2709,9.00,1800000,39,207
9,3303,1,1,1,1,1,1,0,0,1320,8865,2447,12.00,1550000,7,92


In [4]:
# We'll impute missing values using the median for numeric columns and the most
# common value for string columns.
# This is based on some nice code by 'sveitser' at http://stackoverflow.com/a/25562948
from sklearn.base import TransformerMixin
class DataFrameImputer(TransformerMixin):
    def fit(self, X, y=None):
        self.fill = pd.Series([X[c].value_counts().index[0]
            if X[c].dtype == np.dtype('O') else X[c].mean() for c in X],
            index=X.columns)
        return self
    def transform(self, X, y=None):
        return X.fillna(self.fill)

feature_columns_to_use = ['fac_1','fac_2','fac_3','fac_4','fac_5','fac_6','fac_7','fac_8','poi_1','poi_2'
                          ,'poi_3','size','price_monthly','room_count','total_call']
# nonnumeric_columns = ['Sex']

In [5]:
# Join the features from train and test together before imputing missing values,
# in case their distribution is slightly different
big_X = train_df[feature_columns_to_use].append(test_df[feature_columns_to_use])
big_X_imputed = DataFrameImputer().fit_transform(big_X)

In [6]:
# XGBoost doesn't (yet) handle categorical features automatically, so we need to change
# them to columns of integer values.
# See http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing for more
# details and options
# le = LabelEncoder()
# for feature in nonnumeric_columns:
#     big_X_imputed[feature] = le.fit_transform(big_X_imputed[feature])

In [7]:
train_X = big_X_imputed[0:train_df.shape[0]].as_matrix()
test_X = big_X_imputed[train_df.shape[0]::].as_matrix()
train_y = train_df['gender']
# X_train, X_test, Y_train, Y_test = train_test_split(train_X, train_y, test_size=0.3, random_state=101)
train_X

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        1.50000000e+06, 6.00000000e+00, 7.20000000e+01],
       [1.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        1.50000000e+06, 3.00000000e+01, 5.60000000e+01],
       [1.00000000e+00, 6.11674499e-01, 1.00000000e+00, ...,
        1.60000000e+06, 2.00000000e+01, 1.09000000e+02],
       ...,
       [1.00000000e+00, 1.00000000e+00, 1.00000000e+00, ...,
        2.50000000e+06, 1.00000000e+00, 1.80000000e+01],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        4.00000000e+05, 2.10000000e+01, 1.81000000e+02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.50000000e+05, 1.00000000e+01, 6.00000000e+00]])

In [12]:
# You can experiment with many other options here, using the same .fit() and .predict()
# methods; see http://scikit-learn.org
# This example uses the current build of XGBoost, from https://github.com/dmlc/xgboost
gbm = xgb.XGBClassifier(max_depth=5, n_estimators=2900, learning_rate=0.5).fit(train_X, train_y)
# gbm.score(X_test, Y_test)
predictions = gbm.predict(test_X)
predictions

array(['putra', 'putri', 'putri', 'putri', 'putra', 'putri', 'campur',
       'campur', 'campur', 'putri', 'putra', 'putri', 'putri', 'campur',
       'putri', 'putri', 'putri', 'putra', 'putra', 'putri', 'putri',
       'putri', 'putri', 'putri', 'putri', 'putri', 'putri', 'putra',
       'putra', 'putri', 'putra', 'putri', 'putri', 'putri', 'putri',
       'campur', 'putra', 'putri', 'putra', 'putra', 'putra', 'putri',
       'putri', 'campur', 'putri', 'putri', 'putri', 'putri', 'putri',
       'putri', 'putri', 'putra', 'putri', 'putra', 'putri', 'putri',
       'putra', 'campur', 'putra', 'campur', 'putri', 'putra', 'campur',
       'putri', 'putra', 'putri', 'putra', 'putra', 'putra', 'campur',
       'putri', 'putri', 'campur', 'campur', 'campur', 'campur', 'campur',
       'putra', 'putra', 'putra', 'putra', 'putri', 'putri', 'putri',
       'putri', 'putri', 'putri', 'putri', 'putri', 'putri', 'putra',
       'putri', 'putri', 'putri', 'putri', 'putri', 'putri', 'putra',
     

In [13]:
submission = pd.DataFrame({ 'id': test_df['id'],
                            'gender': predictions })
submission.to_csv("submission/33th_submission_XGBboost_transformer.csv", index=False)